In [1]:
import numpy as np
import pygame
import gymnasium as gym
from gymnasium import spaces
import math
from bond_graph import *
from bond_graph_nodes import*
from itertools import permutations
import random
import copy
from gymnasium.envs.registration import register

pygame 2.5.2 (SDL 2.30.2, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [10]:
seed = None
MAX_PARAM_VAL = 10
num_node_types = 6
max_nodes = 8

add_node_space = spaces.Discrete(num_node_types-3, start=3, seed=seed) # node additions correspond to choosing what type you want, don't include the NONE type for adding
add_edge_space = spaces.MultiDiscrete([max_nodes, max_nodes, 2], seed=seed) # edge additions sample space

action_space = spaces.Dict(
    {
        'node_or_bond': spaces.Discrete(2, start=0, seed=seed),
        'node_param': spaces.Discrete(MAX_PARAM_VAL, start=1, seed=seed),
        "node_type": add_node_space,
        "bond": add_edge_space,
    }
)

print(add_node_space.sample())

# print("Action Space: ", action_space)

# flattened_action_space = spaces.utils.flatten_space(action_space)
# print("Flattened Action Space: ", flattened_action_space)
# print(flattened_action_space.shape)

# obs = action_space.sample()
# flat_obs = spaces.utils.flatten(action_space, obs)
# print("Obs: ", obs)
# print("Flat Obs: ", flat_obs)
# print(flat_obs.shape)

# unflattened_obs = spaces.utils.unflatten(action_space, flat_obs)
# print(unflattened_obs)


4


In [3]:
# from gymnasium.spaces import Dict, Discrete, Box
space = spaces.Dict(
    {
        "position": spaces.Discrete(2), 
        "velocity": spaces.Box(0, 1, shape=(2, 2))
    }
) 
obs = space.sample()
print(obs)

flat_space = spaces.utils.flatten_space(space) 
print(flat_space.shape)
flat_obs = spaces.utils.flatten(space, obs)
print(flat_obs)


OrderedDict({'position': 1, 'velocity': array([[0.6317108 , 0.9872838 ],
       [0.4180581 , 0.32717404]], dtype=float32)})
(6,)
[0.         1.         0.63171083 0.98728383 0.4180581  0.32717404]
